# Full estimation with split solar gains

## Import needed libraries

In [1]:
from modestpy import Estimation
import data_importer as data
import fmpy
import pandas as pd
import os
from plotly.subplots import make_subplots
import csv

## Import data
Use the import function for the weather data to import the other data:

<table style="float:left">
<thead>
  <tr>
    <th>Data</th>
    <th>Variable name</th>
    <th>File name</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td>External temperatures</td>
    <td>weather</td>
    <td>weather.csv</td>
  </tr>
  <tr>
    <td>Solar radiation</td>
    <td>solar_rad</td>
    <td>solar_rad.csv</td>
  </tr>
  <tr>
    <td>System temperatures</td>
    <td>riser_CE</td>
    <td>riser_CE.csv</td>
  </tr>
  <tr>
    <td>Occupant count</td>
    <td>occupants</td>
    <td>occupants.csv</td>
  </tr>
  <tr>
    <td>Indoor temperatures</td>
    <td>temps</td>
    <td>temps_CE.csv</td>
  </tr>
</tbody>
</table>

In [2]:
data_path = f"Data/"

# Import external temperature time series from weather data (weather.csv)
weather = pd.read_csv(os.path.join(data_path,"weather.csv"),index_col=0)
weather.index = pd.to_datetime(weather.index)

# Import solar radiation time series (solar_rad.csv)

# Import system temperatures time series (riser_CE.csv)

# Import occupant time series (occupants.csv)

# Import indoor temperature time series (temps_CE.csv)

## Group and clean data
Here we group the data into a variables used by ModestPy. We will not go through these - just execute the cells.

### Inputs

In [ ]:
inp = pd.DataFrame(index = weather.index)
inp["time"] = weather.index

inp["solrad_north"] = solar_rad["north"]
inp["solrad_south"] = solar_rad["south"]
inp["solrad_east"] = solar_rad["east"]
inp["solrad_west"] = solar_rad["west"]

inp["Tout"] = weather["temp"]

inp["Tvestp"] = data.create_ventilation_stp(weather)["Tvest"]+1

inp["Tsup"] = riser_CE["external_temperature_left"]

inp["verate"] = data.create_ventilation_rate(inp.index,4,19)

inp["occ"] = occupants["inhouse"]*0.2107
inp["occ"] = inp["occ"]*2.5

inp = inp.reset_index(drop=True)
dates_inp = inp["time"]
inp["time"] = inp["time"].diff().astype('timedelta64[s]').astype('float').cumsum().fillna(0)
inp = inp.set_index("time")

inp = inp.fillna(method="bfill")

### Ideals

In [ ]:
ideal = pd.DataFrame(index = temps["avg"].index)
ideal["time"] = temps["avg"].index
ideal["T"] = temps["avg"]

ideal["Tret"] = riser_CE["external_temperature_right"]

ideal = ideal.reset_index(drop=True)

ideal["time"] = ideal["time"].diff().astype('timedelta64[s]').astype('float').cumsum().fillna(0)
ideal = ideal.set_index("time")

ideal=ideal.fillna(method="bfill")

## Specify 'est' and 'known'
Here we define the known parameters and bounds for the estimated parameters.

In [ ]:
known = {
         'Vi': 9839,
         'maxVent': 33992*0.226, 
         'CO2n': 415, 
         'Vinf': 100,
         'shgcNorth': 0, 
         'shgcWest': 0
        }

est =   {
         'shgcSouth': [30, 20, 50],
         'shgcEast':  [30, 20, 50],
         'RExt': [2, 0.5, 5],
         'nomPower': [15000, 10000, 50000],
         'maxMassFlow':[1,0.1,2],
         'imass':[14,10,18],
         'Tstp':[23.5,22.8,24]
        }


ic_param = {
    "TAirInit":"T",
    "TretInit":"Tret"
}

## Plot inputs
Plot the inputs in a graph to test if data import is successful.

In [ ]:
fig = make_subplots(rows=6, cols=1,
                    shared_xaxes = True,
                    vertical_spacing=0.05,
                   )
row = 0
# Outside temperature
row += 1
fig.add_scatter(x= dates_inp, y=inp["Tout"],row=row,col=1,name = "Outside - Simulated",legendgroup=str(row))

# Ventilation setpoint
row += 1
fig.add_scatter(x= dates_inp, y=inp["Tvestp"],row=row,col=1,name = "Ventilation stp",legendgroup=str(row))

# Solar gains
row += 1
fig.add_scatter(x= dates_inp, y=inp["solrad_north"],row=row,col=1,name = "Solar north",legendgroup=str(row))
fig.add_scatter(x= dates_inp, y=inp["solrad_south"],row=row,col=1,name = "Solar south",legendgroup=str(row))
fig.add_scatter(x= dates_inp, y=inp["solrad_east"],row=row,col=1,name = "Solar east",legendgroup=str(row))
fig.add_scatter(x= dates_inp, y=inp["solrad_west"],row=row,col=1,name = "Solar west",legendgroup=str(row))

# Supply temperatures
row += 1
fig.add_scatter(x= dates_inp, y=inp["Tsup"],row=row,col=1,name = "Supply - Measured",legendgroup=str(row))

# Occupancy
row += 1
fig.add_scatter(x= dates_inp, y=inp["occ"],row=row,col=1,name = "Occupancy",legendgroup=str(row))

# verate
row += 1
fig.add_scatter(x= dates_inp, y=inp["verate"],row=row,col=1,name = "verate",legendgroup=str(row))

fig.update_layout(
    # width=800,
    height=250*row,
    legend_tracegroupgap = 250*245/300,)

fig.update_xaxes()
fig.show()

## Estimation with ModestPy
Use the following parameters to setup and start an estimation case study, based on the previous parameter estimation example:

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
</style>
<table class="tg" style="float:left">
<thead>
  <tr>
    <th class="tg-0pky">Variable</th>
    <th class="tg-0pky">Value</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0pky">ic_param</td>
    <td class="tg-0pky">ic_param</td>
  </tr>
  <tr>
    <td class="tg-0pky">lp_n</td>
    <td class="tg-0pky">1</td>
  </tr>
  <tr>
    <td class="tg-0pky">lp_frame</td>
    <td class="tg-0pky">(start, end)</td>
  </tr>
  <tr>
    <td class="tg-0pky">vp</td>
    <td class="tg-0pky">(end, end + validation)</td>
  </tr>
  <tr>
    <td class="tg-0pky">ftype</td>
    <td class="tg-0pky">'NRMSE'</td>
  </tr>
  <tr>
    <td class="tg-0pky">default_log</td>
    <td class="tg-0pky">True</td>
  </tr>
  <tr>
    <td class="tg-0pky">logfile</td>
    <td class="tg-0pky">'simple.log'</td>
  </tr>
  <tr>
    <td class="tg-0pky">methods</td>
    <td class="tg-0pky">('MODESTGA',)</td>
  </tr>
  <tr>
    <td class="tg-0pky">generations</td>
    <td class="tg-0pky">40</td>
  </tr>
  <tr>
    <td class="tg-0pky">pop_size</td>
    <td class="tg-0pky">40</td>
  </tr>
  <tr>
    <td class="tg-0pky">trm_size</td>
    <td class="tg-0pky">7</td>
  </tr>
  <tr>
    <td class="tg-0pky">tol</td>
    <td class="tg-0pky">1e-3</td>
  </tr>
  <tr>
    <td class="tg-0pky">workers</td>
    <td class="tg-0pky">3</td>
  </tr>
</tbody>
</table>

In [ ]:
# Parameters
fmu_path = "FMUs/LBForsikring_ZoneR2C2StpRadiatorSolarSplit.fmu"
workdir = f"Results/"

if not os.path.exists(workdir):
  os.makedirs(workdir)

start = 0              # Start of learning period
end = 3600*24*14       # End of learning period
validation = 3600*24*7 # Length of validation period

In [ ]:
# Setup estimation


In [ ]:
# Run estimation


In [ ]:
# Validate solution


In [ ]:
# Print estimates


## Simulate, analyze and plot estimated solution
Run a simulation for the entire data period and plot the results

In [ ]:
# Rewrite inputs to FMPy format
inp_copy = inp.copy()
inp_np = inp_copy.to_records()
wanted_outputs = ["T","Tout","Tret","Tsup","P_temp.y","heatFlowSensor.Q_flow","prescribedHeatFlow.Q_flow","re.port_b.Q_flow","occ","Tstp","filter1.y","verate","filter2.y","solarGainNorth.Q_flow","solarGainSouth.Q_flow","solarGainEast.Q_flow","solarGainWest.Q_flow","rad.Q_flow","airMix.port_b.Q_flow","rad.m_flow","max1.y"]

# Combine known and estimates to 'values' variable
est_path = f"Results/final.csv"
if 'estimates' in locals():
    values = {**known, **estimates}
else:
    reader = csv.DictReader(open(est_path))
    dictobj = next(reader) 
    dictobj
    values = {**known,
              **dictobj}
    
start_time = 0
stop_time = inp.index[-1]
result_realistic=fmpy.simulate_fmu(fmu_path,input=inp_np,start_values=values,output = wanted_outputs,start_time = start_time,stop_time=stop_time,relative_tolerance=1e-5)

### Clean and analyse data

In [ ]:
result_df = pd.DataFrame.from_records(result_realistic, index = "time")
measured_df = pd.concat((ideal,inp),axis=1)

result_df.index = result_df.index/3600
measured_df.index = measured_df.index/3600

result_df["TimeDelta"] = result_df.index.to_series().diff() # Timestep in Hours
measured_df["TimeDelta"] = measured_df.index.to_series().diff() # Timestep in Hours

result_df["total_heating"] = result_df["TimeDelta"]*result_df["rad.Q_flow"]*(-1)/1000

# Function to get a weighted average
def w_avg(df, values, weights):
    d = df[values]
    w = df[weights]
    return (d * w).sum() / w.sum()

sup_temp_w_avg  = w_avg(result_df,"Tsup","total_heating")
ret_temp_w_avg  = w_avg(result_df,"Tret","total_heating")


print("Total heating:",result_df["total_heating"].sum().round(0),"kWh")
print("Avg. supply temp:",sup_temp_w_avg.round(1),"°C")
print("Avg. return temp:",ret_temp_w_avg.round(1),"°C")

### Plot simulation results

In [ ]:
dates_result = pd.to_timedelta(result_df.index,'h')+dates_inp[0]

fig = make_subplots(rows=7, cols=1,
                    shared_xaxes = True,
                    vertical_spacing=0.05,
                    # subplot_titles = ["Room temperature","System temperatures","Occupancy","Solar gains","TRV","Ventilation","Damping","verate"]
                   )


# Outside temperature
row = 1
fig.add_scatter(x= dates_result, y=result_df["Tout"],row=row,col=1,name = "Outside temperature",legendgroup=str(row))
fig.add_scatter(x= dates_result, y=result_df["max1.y"]-273.15,row=row,col=1,name = "Ventilation temperature",legendgroup=str(row))

# Room temperature
row += 1
fig.add_scatter(x= dates_result, y=result_df["T"],row=row,col=1,name = "Room - Simulated",legendgroup=str(row))
fig.add_scatter(x= dates_inp, y=measured_df["T"],row=row,col=1,name = "Room - Measured",legendgroup=str(row))

# System temperatures
row += 1
fig.add_scatter(x= dates_result, y=result_df["Tsup"],row=row,col=1,name = "Supply - Measured",legendgroup=str(row))
fig.add_scatter(x= dates_inp, y=measured_df["Tret"],row=row,col=1,name = "Return - Measured",legendgroup=str(row))
fig.add_scatter(x= dates_result, y=result_df["Tret"],row=row,col=1, name = "Return - Simulated",legendgroup=str(row))


# Occupancy
row += 1
fig.add_scatter(x= dates_inp, y=measured_df["occ"],row=row,col=1,name = "Occupancy",legendgroup=str(row))


# radiator heat
row += 1
fig.add_scatter(x= dates_result, y=result_df["rad.Q_flow"],row=row,col=1,name = "rad.Q_flow",legendgroup=str(row))

# Solar gains
row += 1
fig.add_scatter(x= dates_result, y=result_df["solarGainSouth.Q_flow"],row=row,col=1,name = "Solar gains South",legendgroup=str(row))
fig.add_scatter(x= dates_result, y=result_df["solarGainEast.Q_flow"],row=row,col=1,name = "Solar gains East",legendgroup=str(row))

# ventilation gain
row += 1
fig.add_scatter(x= dates_result, y=result_df["airMix.port_b.Q_flow"],row=row,col=1,name = "Ventilation loss",legendgroup=str(row))
fig.add_scatter(x= dates_result, y=result_df["re.port_b.Q_flow"],row=row,col=1,name = "Transmission loss",legendgroup=str(row))

fig.update_layout(
    height=250*row,
    legend_tracegroupgap = 250*240/300,)

fig.show()

In [ ]:
# Rewrite inputs to FMPy format
inp_low_supply = inp.copy()
inp_low_supply["Tsup"] = inp_low_supply["Tsup"] - 10
inp_np = inp_low_supply.to_records()

In [ ]:
wanted_outputs = ["T","Tout","Tret","Tsup","P_temp.y","heatFlowSensor.Q_flow","prescribedHeatFlow.Q_flow","re.Q_flow","occ","Tstp","filter1.y","verate","filter2.y","solarGainNorth.Q_flow","solarGainSouth.Q_flow","solarGainEast.Q_flow","solarGainWest.Q_flow","rad.Q_flow","airMix.port_b.Q_flow","rad.m_flow","max1.y"]

In [ ]:
# 
if 'estimates' in locals():
    values_low_supply = {**known, **estimates}
else:
    print("estimates are unknown!")

values_low_supply["nomPower"] = 22000
values_low_supply

In [ ]:
start_time = 0
stop_time = inp.index[-1]
result=fmpy.simulate_fmu(fmu_path,input=inp_np,start_values=values_low_supply,output = wanted_outputs,start_time = start_time,stop_time=stop_time,relative_tolerance=1e-5,output_interval=600)
# fmpy.plot_result(result)
# pd.DataFrame.from_records(result, index = "time")

In [ ]:
def w_avg(df, values, weights):
    d = df[values]
    w = df[weights]
    return (d * w).sum() / w.sum()

In [ ]:
result_df_low_supply = pd.DataFrame.from_records(result, index = "time")
measured_df_low_supply = pd.concat((ideal,inp),axis=1)

result_df_low_supply["TimeDelta"] = result_df_low_supply.index.to_series().diff()/3600 # Timestep in Hours
measured_df_low_supply["TimeDelta"] = measured_df_low_supply.index.to_series().diff()/3600 # Timestep in Hours

result_df_low_supply["total_heating"] = result_df_low_supply["TimeDelta"]*result_df_low_supply["rad.Q_flow"]*(-1)/1000

sup_temp_w_avg  = w_avg(result_df_low_supply,"Tsup","total_heating")
ret_temp_w_avg  = w_avg(result_df_low_supply,"Tret","total_heating")


print("Total heating:",result_df_low_supply["total_heating"].sum().round(0),"kWh")
print("Avg. supply temp:",sup_temp_w_avg.round(1),"°C")
print("Avg. supply temp:",ret_temp_w_avg.round(1),"°C")

# result_df_low_supply.head()

In [ ]:
fig = make_subplots(rows=4, cols=1,
                    shared_xaxes = True,
                    vertical_spacing=0.05,
                    # subplot_titles = ["Room temperature","System temperatures","Occupancy","Solar gains","TRV","Ventilation","Damping","verate"]
                   )


# Outside temperature
row = 1
fig.add_scatter(x= result_df_low_supply.index, y=result_df_low_supply["Tout"],row=row,col=1,name = "Outside temperature",legendgroup=str(row))
fig.add_scatter(x= measured_df.index, y=measured_df["Tvestp"],row=row,col=1,name = "Ventilation temperature",legendgroup=str(row))

# Room temperature
row += 1
fig.add_scatter(x= result_df_low_supply.index, y=result_df_low_supply["T"],row=row,col=1,name = "Room - Simulated optimum",legendgroup=str(row))
fig.add_scatter(x= measured_df.index, y=measured_df["T"],row=row,col=1,name = "Room - Measured",legendgroup=str(row))
fig.add_scatter(x= result_df.index, y=result_df["T"],row=row,col=1,name = "Room - Simulated realistic",legendgroup=str(row))

# System temperatures
row += 1
fig.add_scatter(x= result_df_low_supply.index, y=result_df_low_supply["Tsup"],row=row,col=1,name = "Supply - Measured",legendgroup=str(row))
fig.add_scatter(x= measured_df.index, y=measured_df["Tret"],row=row,col=1,name = "Return - Measured",legendgroup=str(row))
fig.add_scatter(x= result_df.index, y=result_df["Tret"],row=row,col=1, name = "Return - Simulated",legendgroup=str(row))
fig.add_scatter(x= result_df_low_supply.index, y=result_df_low_supply["Tret"],row=row,col=1, name = "Return - Simulated optimal",legendgroup=str(row))


# radiator heat
row += 1
fig.add_scatter(x= result_df_low_supply.index, y=result_df_low_supply["rad.Q_flow"],row=row,col=1,name = "rad.Q_flow",legendgroup=str(row))


fig.update_layout(
    # width=800,
    height=250*row,
    legend_tracegroupgap = 250*240/300,)


# fig.update_xaxes(range = [start_time,stop_time])
fig.show()

In [ ]:
# Rewrite inputs to FMPy format
inp_low_supply = inp.copy()
inp_low_supply["Tsup"] = inp_low_supply["Tsup"] - 10
inp_np = inp_low_supply.to_records()

In [ ]:
wanted_outputs = ["T","Tout","Tret","Tsup","P_temp.y","heatFlowSensor.Q_flow","prescribedHeatFlow.Q_flow","re.Q_flow","occ","Tstp","filter1.y","verate","filter2.y","solarGainNorth.Q_flow","solarGainSouth.Q_flow","solarGainEast.Q_flow","solarGainWest.Q_flow","rad.Q_flow","airMix.port_b.Q_flow","rad.m_flow"]

In [ ]:
# 
est_path = f"Results/{month}/final.csv"
if 'estimates' in locals():
    values = {**known, **estimates}
else:
    reader = csv.DictReader(open(est_path))
    dictobj = next(reader) 
    dictobj
    values = {**known,
              **dictobj}

    
values["TAirInit"] = ideal["T"][0]
values["TretInit"] = ideal["Tret"][0]
values

In [ ]:
start_time = 0
stop_time = inp.index[-1]
result=fmpy.simulate_fmu(fmu_path,input=inp_np,start_values=values,output = wanted_outputs,start_time = start_time,stop_time=stop_time,relative_tolerance=1e-5,output_interval=600)
# fmpy.plot_result(result)
# pd.DataFrame.from_records(result, index = "time")

In [ ]:
def w_avg(df, values, weights):
    d = df[values]
    w = df[weights]
    return (d * w).sum() / w.sum()

In [ ]:
result_df_low_supply = pd.DataFrame.from_records(result, index = "time")
measured_df_low_supply = pd.concat((ideal,inp),axis=1)
result_df_low_supply.index = result_df_low_supply.index/3600
measured_df_low_supply.index = measured_df_low_supply.index/3600

result_df_low_supply["TimeDelta"] = result_df_low_supply.index.to_series().diff() # Timestep in Hours
measured_df_low_supply["TimeDelta"] = measured_df_low_supply.index.to_series().diff() # Timestep in Hours

result_df_low_supply["total_heating"] = result_df_low_supply["TimeDelta"]*result_df_low_supply["rad.Q_flow"]*(-1)/1000

sup_temp_w_avg  = w_avg(result_df_low_supply,"Tsup","total_heating")
ret_temp_w_avg  = w_avg(result_df_low_supply,"Tret","total_heating")


print("Total heating:",result_df_low_supply["total_heating"].sum().round(0),"kWh")
print("Avg. supply temp:",sup_temp_w_avg.round(1),"°C")
print("Avg. supply temp:",ret_temp_w_avg.round(1),"°C")

# result_df_low_supply.head()

In [ ]:

cm = 1/2.54

fig, axes = plt.subplots(nrows=2, ncols=1,
                    sharex=True,
                    # gridspec_kw={'height_ratios': [2, 3]},
                    figsize=(17*cm, 10*cm))

# result_df_low_supply["T"].plot(label="Simulated T", ax=axes[0])
result_df_low_supply["T"].plot(label="Simulated T", ax=axes[0])
measured_df["T"].plot(label="Measured T",ax=axes[0])
axes[0].set_ylabel("$T_{room}$ [°C]")


result_df_low_supply["Tret"].plot(label="Simulated Tret", ax=axes[1])
measured_df["Tret"].plot(label="Measured Tret",ax=axes[1])
axes[1].set_ylabel("$T_{ret}$ [°C]")
axes[1].set_xlabel("Time [h]")


fig.legend(["Low Tsup","Measured"],loc='lower right', bbox_to_anchor=(0.5, 0.12, 0.42, 0.5))

plt.savefig("test_low_tsup_estimated.pdf")

In [ ]:
# fig = make_subplots(rows=4, cols=1,
#                     shared_xaxes = True,
#                     vertical_spacing=0.05,
#                     # subplot_titles = ["Room temperature","System temperatures","Occupancy","Solar gains","TRV","Ventilation","Damping","verate"]
#                    )


# # Outside temperature
# row = 1
# fig.add_scatter(x= result_df_low_supply.index, y=result_df_low_supply["Tout"],row=row,col=1,name = "Outside temperature",legendgroup=str(row))
# fig.add_scatter(x= measured_df.index, y=measured_df["Tvestp"],row=row,col=1,name = "Ventilation temperature",legendgroup=str(row))

# # Room temperature
# row += 1
# fig.add_scatter(x= result_df_low_supply.index, y=result_df_low_supply["T"],row=row,col=1,name = "Room - Simulated optimum",legendgroup=str(row))
# fig.add_scatter(x= measured_df.index, y=measured_df["T"],row=row,col=1,name = "Room - Measured",legendgroup=str(row))
# fig.add_scatter(x= result_df.index, y=result_df["T"],row=row,col=1,name = "Room - Simulated realistic",legendgroup=str(row))

# # System temperatures
# row += 1
# fig.add_scatter(x= result_df_low_supply.index, y=result_df_low_supply["Tsup"],row=row,col=1,name = "Supply - Measured",legendgroup=str(row))
# fig.add_scatter(x= measured_df.index, y=measured_df["Tret"],row=row,col=1,name = "Return - Measured",legendgroup=str(row))
# fig.add_scatter(x= result_df.index, y=result_df["Tret"],row=row,col=1, name = "Return - Simulated",legendgroup=str(row))
# fig.add_scatter(x= result_df_low_supply.index, y=result_df_low_supply["Tret"],row=row,col=1, name = "Return - Simulated optimal",legendgroup=str(row))


# # radiator heat
# row += 1
# fig.add_scatter(x= result_df_low_supply.index, y=result_df_low_supply["rad.Q_flow"],row=row,col=1,name = "rad.Q_flow",legendgroup=str(row))


# fig.update_layout(
#     # width=800,
#     height=250*row,
#     legend_tracegroupgap = 250*240/300,)


# # fig.update_xaxes(range = [start_time,stop_time])
# fig.show()